In [1]:
import pandas as pd
import numpy as np

In [2]:
def esperanza_get_id(check_dict):
    if pd.isnull(check_dict):
        return check_dict
    return check_dict['id']

def esperanza_get_name(check_dict):
    if pd.isnull(check_dict):
        return check_dict
    return check_dict['name']

In [3]:
def esperanza_get_keyword_id(check_dict, keyword):
    if pd.isnull(check_dict):
        return check_dict
    else:
        return esperanza_get_id(check_dict.get(keyword))
    
def esperanza_get_keyword_name(check_dict, keyword):
    if pd.isnull(check_dict):
        return check_dict
    else:
        return esperanza_get_name(check_dict.get(keyword))

In [4]:
def esperanza_coord_x(coord_list):
    if len(coord_list) == 0:
        return coord_list
    return coord_list.str[0]

def esperanza_coord_y(coord_list):
    if len(coord_list) == 0:
        return coord_list
    return coord_list.str[1]

def esperanza_coord_z(coord_list):
    if len(coord_list) == 0:
        return coord_list
    return coord_list.str[2]

In [5]:
def esperanza_get_attribute(check_dict, attr):
    if pd.isnull(check_dict):
        return check_dict
    return check_dict[attr]

In [6]:
def esperanza_clean_basic(df, col_name):
    col = df.get(col_name)
    ids = col.transform(esperanza_get_id)
    names = col.transform(esperanza_get_name)
    df[col_name + "_id"] = ids
    df[col_name + "_name"] = names
    return df.drop(col_name, axis = 1)

In [7]:
def esperanza_clean_basic_2(df, col_name, sub_name):
    col = df.get(col_name)
    ids = col.apply(esperanza_get_keyword_id, args = (sub_name, ))
    names = col.apply(get_keyword_name, args = (sub_name, ))
    df[col_name + "_" + sub_name + "_id"] = ids
    df[col_name + "_" + sub_name + "_name"] = names
    return df
    

In [9]:
# this will return a cleaned dataframe for all of these columns:
# type, possession_team, play_pattern, team, player, position, interception, substitution, bad_behaviour, and pass
def esperanza_clean_total(df):
    clean_basic_list = ['type', 'possession_team', 'play_pattern', 'team', 'player', 'position']
    for col in clean_basic_list:
        df = esperanza_clean_basic(df, col)
        
    clean_basic_list_2 = ['interception', 'substitution', 'bad_behaviour', 'pass']
    for col in clean_basic_list_2:
        if col == 'interception':
            df = esperanza_clean_basic_2(df, col, 'outcome')

        if col == 'substitution':
            df = esperanza_clean_basic_2(df, col, 'outcome')
            df = esperanza_clean_basic_2(df, col, 'replacement')

        if col == 'bad_behaviour':
            df = esperanza_clean_basic_2(df, col, 'card')

        if col == 'pass':
            ser = df.get(col)
            df[col + '_length'] = ser.apply(esperanza_get_attribute, args = ('length', ))
            df[col + '_angle'] = ser.apply(esperanza_get_attribute, args = ('angle', ))
            df = esperanza_clean_basic_2(df, col, 'recipient')
            df = esperanza_clean_basic_2(df, col, 'height')
            df = esperanza_clean_basic_2(df, col, 'type')
            df = esperanza_clean_basic_2(df, col, 'body_part')
            df[col + '_end_location_x'] = ser.apply(esperanza_get_attribute, args = ('end_location', )).transform(esperanza_coord_x)
            df[col + '_end_location_y'] = ser.apply(esperanza_get_attribute, args = ('end_location', )).transform(esperanza_coord_y)


        df = df.drop(col, axis = 1)
        
    locations = df.get("location")
    df["location_x"] = locations.transform(coord_x)
    df["location_y"] = locations.transform(coord_y)
    df = df.drop('location', axis = 1)
    
    return df

In [10]:
# need to run this carefully because the table returned should be kept separately from other files here

def esperanza_clean_related_events(df):
    reduced = df[["id", "related_events"]]
    related = reduced.explode('related_events').reset_index(drop = True)
    # all this does is have two columns - id and related events as a list - not sure if anything else is wanted
    
    return related